In [1]:
%pip install git+https://github.com/openai/swarm.git

  Cloning https://github.com/openai/swarm.git to /tmp/pip-req-build-0lom2oyz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /tmp/pip-req-build-0lom2oyz
  Resolved https://github.com/openai/swarm.git to commit 0c82d7d868bb8e2d380dfd2a319b5c3a1f4c0cb9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.0/221.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 30.7 MB/s eta 0:00:00
  Created wheel for swarm: filename=swarm-0.1.0-py3-none-any.whl size=25913 sha256=b0fcc9720f980e

swarm is used here to create and manage multiple AI agents that work together on tasks. The code later relies on it to define the different roles in the blog creation workflow.

In [2]:
def complete_blog_post(title, content):
  filename = title.lower().replace(" ", "-") + ".md"

  with open(filename, "w", encoding="utf-8") as file:
    file.write(content)

  print("Blog post creation completed")
  return "Task completed"

This is the final step in the AI agent workflow — after editing is complete, this function writes the polished content into a file so it’s stored permanently.

In [3]:
from swarm import Agent

def admin_instructions(context_variables):
  topic = context_variables.get("topic", "No topic provided")
  return f"""You are the Admin Agent overseeing the blog post project on the topic: '{topic}'.
Your responsibilities include initiating the project, providing guidance, and reviewing the final content.
Once you've set the topic, call the function to transfer to the planner agent."""

def planner_instructions(context_variables):
  topic = context_variables.get("topic", "No topic provided")
  return f"""You are the Planner Agent. Based on the following topic: '{topic}'
Organize the content into topics and sections with clear headings that will each be individually researched as points in the greater blog post.
Once the outline is ready, call the researcher agent."""

def researcher_instructions(context_variables):
  return """You are the Researcher Agent. your task is to provide dense context and information on the topics outlined by the previous planner agent.
This research will serve as the information that will be formatted into a body of a blog post. Provide comprehensive research like notes for each of the sections outlined by the planner agent.
Once your research is complete, transfer to the writer agent"""

def writer_instructions(context_variables):
  return """You are the Writer Agent. using the prior information write a clear blog post following the outline from the planner agent.
    Summarise and include as much information relevant from the research into the blog post.
    The blog post should be quite large as the context the context provided should be quite dense.
Write clear, engaging content for each section.
Once the draft is complete, call the function to transfer to the Editor Agent."""

def editor_instructions(context_variables):
  return """You are the Editor Agent. Review and edit the prior blog post completed by the writer agent.
Make necessary corrections and improvements.
Once editing is complete, call the function to complete the blog post"""

def transfer_to_planner():
  return planner_agent

def transfer_to_researcher():
  return researcher_agent

def transfer_to_writer():
  return writer_agent

def transfer_to_editor():
  return editor_agent

admin_agent = Agent(
    name = "Admin Agent",
    instructions = admin_instructions,
    functions = [transfer_to_planner],
)

planner_agent = Agent(
    name = "Planner Agent",
    instructions = planner_instructions,
    functions = [transfer_to_researcher],
)

researcher_agent = Agent(
    name = "Researcher Agent",
    instructions = researcher_instructions,
    functions = [transfer_to_writer],
)

writer_agent = Agent(
    name = "Writer Agent",
    instructions = writer_instructions,
    functions = [transfer_to_editor],
)

editor_agent = Agent(
    name = "Editor Agent",
    instructions = editor_instructions,
    functions = [complete_blog_post],
)


This cell builds the multi-agent workflow pipeline where each agent plays a specific role, and control flows from one agent to the next until the blog is done.

In [4]:
OPENAI_API_KEY= 'sk-ZuzGDRSxDpL_S3JjZpz4WClilSQaJP7T9s-C2YUOYFT3BlbkFJKaQ25y_6YZ4W_Ld1kGHki1dpvhHYJjKHNAnxTVRmMA'

In [5]:
from swarm.repl import run_demo_loop
import os

def run():
  os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY
  run_demo_loop(admin_agent, debug=True)

In [6]:
run()

Starting Swarm CLI 🐝
User: Neural Network vs LSTM
[2025-08-14 17:22:17] Getting chat completion for...: [{'role': 'system', 'content': "You are the Admin Agent overseeing the blog post project on the topic: 'No topic provided'.\nYour responsibilities include initiating the project, providing guidance, and reviewing the final content.\nOnce you've set the topic, call the function to transfer to the planner agent."}, {'role': 'user', 'content': 'Neural Network vs LSTM'}]
[2025-08-14 17:22:18] Received completion: ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_dweR6dc8MTgvwLtk8PGIT8jm', function=Function(arguments='{}', name='transfer_to_planner'), type='function')])
[2025-08-14 17:22:18] Processing tool call: transfer_to_planner with arguments {}
[2025-08-14 17:22:18] Getting chat completion for...: [{'role': 'system', 'content': "You are the Planner Agent. Base

KeyboardInterrupt: Interrupted by user